<a href="https://colab.research.google.com/github/bluevit/Recruit-Right-Precision-Hiring-with-AI-Insight/blob/main/org_NER_for_ResumeAnalysis_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Training spaCy NER for Resume Analysis</h3>


In [ ]:
!pip install numpy #==1.25

In [ ]:
!pip install -U spacy #==3.7.2
!pip install spacy-transformers

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf  # Download Transformer-based model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install -U spacy-transformers
import spacy
from spacy.tokens import DocBin

In [ ]:
config_text = """[paths]
train = "./train_data.spacy"
dev = "./dev_data.spacy"
vectors = null

[nlp]
lang = "en"
pipeline = ["transformer", "ner"]

[components]
[components.ner]
factory = "ner"

[components.transformer]
factory = "transformer"

[training]
dropout = 0.2
patience = 5
max_epochs = 20
seed = 42
gpu_allocator = "pytorch"
accumulate_gradient = 1
max_steps = 20000
eval_frequency = 200
frozen_components = []
annotating_components = []
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
before_to_disk = null
before_update = null

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[training.optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.00005

[training.score_weights]
"""

with open("config.cfg", "w") as f:
    f.write(config_text)

print("✅ Final corrected config.cfg written.")


✅ Final corrected config.cfg written.


In [ ]:
!cat config.cfg


[paths]
train = "./train_data.spacy"
dev = "./dev_data.spacy"
vectors = null

[nlp]
lang = "en"
pipeline = ["transformer", "ner"]

[components]
[components.ner]
factory = "ner"

[components.transformer]
factory = "transformer"

[training]
dropout = 0.2
patience = 5
max_epochs = 20
seed = 42
gpu_allocator = "pytorch"
accumulate_gradient = 1
max_steps = 20000
eval_frequency = 200
frozen_components = []
annotating_components = []
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
before_to_disk = null
before_update = null

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[training.optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.00005

[training.score_weights]


In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

# !git clone https://github.com/Mehyarmlaweh/NER-Annotated-CVs.git


fatal: destination path 'CV-Parsing-using-Spacy-3' already exists and is not an empty directory.


In [ ]:
import json
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [ ]:
import json
import spacy
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load training data JSON again
with open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r', encoding='utf-8') as f:
    cv_data = json.load(f)

# Split into train/dev
train_data, dev_data = train_test_split(cv_data, test_size=0.1, random_state=42)


In [ ]:
# def convert_to_spacy(data, filename):
#     nlp = spacy.blank("en")
#     db = DocBin()
#     skipped = 0

#     for text, annot in tqdm(data):
#         doc = nlp.make_doc(text)
#         ents = []
#         seen_tokens = set()
#         valid = True

#         for start, end, label in annot["entities"]:
#             if start >= end or start < 0 or end > len(text):
#                 valid = False
#                 break

#             span = doc.char_span(start, end, label=label, alignment_mode="contract")
#             if span is None:
#                 valid = False
#                 break

#             if any(tok.i in seen_tokens for tok in span):
#                 valid = False
#                 break

#             ents.append(span)
#             seen_tokens.update([tok.i for tok in span])

#         if valid:
#             try:
#                 doc.ents = ents
#                 db.add(doc)
#             except Exception:
#                 skipped += 1
#         else:
#             skipped += 1

#     db.to_disk(filename)
#     print(f"✅ Saved {len(data)-skipped} examples to {filename}. Skipped {skipped}.")


def convert_to_spacy(data, filename):
    import spacy
    from spacy.tokens import DocBin
    from tqdm import tqdm

    nlp = spacy.blank("en")
    db = DocBin()
    skipped = 0

    for text, annot in tqdm(data):
        text = text.strip()  # clean whitespace
        doc = nlp.make_doc(text)
        ents = []
        seen_tokens = set()
        valid = True

        for start, end, label in annot["entities"]:
            if start >= end or start < 0 or end > len(text):
                valid = False
                break

            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None or span.text.strip() != span.text:
                valid = False
                break

            if any(tok.i in seen_tokens for tok in span):
                valid = False
                break

            ents.append(span)
            seen_tokens.update([tok.i for tok in span])

        if valid:
            try:
                doc.ents = ents
                db.add(doc)
            except Exception:
                skipped += 1
        else:
            skipped += 1

    db.to_disk(filename)
    print(f"✅ Saved {len(data)-skipped} examples to {filename}. Skipped {skipped}.")


In [ ]:
convert_to_spacy(train_data, "train_data.spacy")
convert_to_spacy(dev_data, "dev_data.spacy")


100%|██████████| 180/180 [00:01<00:00, 118.48it/s]


✅ Saved 81 examples to train_data.spacy. Skipped 99.


100%|██████████| 20/20 [00:00<00:00, 94.21it/s]

✅ Saved 8 examples to dev_data.spacy. Skipped 12.


In [ ]:
len(cv_data)

200

In [ ]:
# you can see the data(json) by uncommenting below statement
#cv_data

In [ ]:
#!python -m spacy train config.cfg --output ./output --gpu-id 0

!python -m spacy train config.cfg --output ./output --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
2025-05-22 06:32:26.548078: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 06:32:26.566420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747895546.588124   23092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747895546.594666   23092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 0

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


---
config.cfg must be modified in order to reduce the training time<br>
(generated in *content/CV-Parsing-using-Spacy-3/data/training/config.cfg*)<br><br>
max_steps parameter must be set to some low number like 2000 or 4000, which will drastically reduce the training time.</h3>
<hr>

This function takes a dataset of text and annotations, processes each text to identify and label entities, and stores the results in a DocBin object that can be used for further training or analysis with spaCy.

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + ' ' + str(text) + '\n'
        file.write(err_data)
      else :
        ents.append(span)
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
      #print(text)
  return db

In [ ]:
# !pip install optuna
# import optuna

# def objective(trial):
#     dropout = trial.suggest_float("dropout", 0.1, 0.3)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     # Update config.cfg
#     with open("/content/CV-Parsing-using-Spacy-3/data/training/config.cfg", "r") as file:
#         config = file.read()

#     config = config.replace("dropout = 0.2", f"dropout = {dropout}")
#     config = config.replace("batch_size = 32", f"batch_size = {batch_size}")

#     with open("/content/CV-Parsing-using-Spacy-3/data/training/config.cfg", "w") as file:
#         file.write(config)

#     # Train the model
#     !python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --gpu-id 0

#     # Check if the model was created
#     import os
#     if os.path.exists("./output/model-best"):  # Check if the model directory exists
#         # Load trained model and evaluate
#         nlp = spacy.load("./output/model-best")
#         test_text = "John Doe worked at Microsoft as a Data Scientist from 2019-2023."
#         doc = nlp(test_text)

#         return len(doc.ents)  # Optimize based on entity count
#     else:
#         print("Model not found at ./output/model-best. Check training output.")
#         return 0  # Return a default value if the model is not found

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=5)

# print("Best Hyperparameters:", study.best_params)

**Training**

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
file = open('error.txt', 'w')
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

100%|██████████| 60/60 [00:00<00:00, 78.35it/s]


In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
2025-05-22 06:35:15.206496: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 06:35:15.224643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747895715.246514   23902 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747895715.253145   23902 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 0

<h3>Testing the Trained Model</h3>

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.0 MB/s eta 0:00:00


In [ ]:
# from pdfminer.high_level import extract_text

# text = extract_text("/content/M_Abdullah_Resume.pdf")
# print(text)

In [ ]:
# doc = nlp(text)
# for ent in doc.ents:
#   print(ent.text, '---->' , ent.label_)

testing


In [ ]:
import spacy

# Load your trained model
nlp = spacy.load("output/model-last")

In [ ]:
# # Read resume text
# from pdfminer.high_level import extract_text
# text = extract_text("/content/M_Abdullah_Resume.pdf")

In [ ]:
# # Process with NER model
# doc = nlp(text)
# for ent in doc.ents:
#     print(ent.text, "---->", ent.label_)

In [ ]:
# import spacy
# from spacy.matcher import Matcher
# import re
# from pdfminer.high_level import extract_text

# # Load the trained model
# nlp = spacy.load("output/model-last")

# # Extract text from the new resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary to remove duplicates
# extracted_data = {"Name": set(), "Degree": set(), "College Name": set(), "Location": set(), "Skills": set()}

# # Extract NER results
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract experience using regex
# exp_pattern = re.findall(r'(\d+)\+?\s*years?\s*of\s*experience', text, re.IGNORECASE)
# if exp_pattern:
#     extracted_data["Experience"] = {exp_pattern[0] + " years"}

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)

# # Display the cleaned-up results
# print("---- Cleaned Entity Recognition Output ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {', '.join(values)}")


In [ ]:
# import spacy
# from spacy.matcher import Matcher
# import re
# from pdfminer.high_level import extract_text

# # Load the trained model
# nlp = spacy.load("output/model-last")

# # Extract text from the new resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary to remove duplicates
# extracted_data = {"Name": set(), "Degree": set(), "College Name": set(), "Location": set(), "Skills": set()}

# # Extract NER results
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract experience using regex
# exp_pattern = re.findall(r'(\d+)\+?\s*years?\s*of\s*experience', text, re.IGNORECASE)
# if exp_pattern:
#     extracted_data["Experience"] = {exp_pattern[0] + " years"}

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)

# # Display the cleaned-up results
# print("---- Cleaned Entity Recognition Output ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {', '.join(values)}")


In [ ]:
# import spacy
# import re
# from pdfminer.high_level import extract_text
# from dateutil import parser
# from datetime import datetime

# # Load the trained NER model
# nlp = spacy.load("output/model-last")

# # Extract text from the resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary
# extracted_data = {
#     "Name": set(),
#     "Degree": set(),
#     "College Name": set(),
#     "Location": set(),
#     "Skills": set(),
#     "Experience": set(),
# }

# # Extract Named Entities (NER)
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract job experience details (using regex)
# experience_pattern = re.findall(
#     r"([\w\s\-/]+)\s*[\n]*\s*(\w+\s*\d{4})\s*–\s*(\w+\s*\d{4}|Present)", text
# )

# total_experience = 0
# for job_title, start_date, end_date in experience_pattern:
#     try:
#         start_date = parser.parse(start_date)
#         end_date = (
#             parser.parse(end_date) if "Present" not in end_date else datetime.today()
#         )
#         experience_months = (end_date.year - start_date.year) * 12 + (
#             end_date.month - start_date.month
#         )
#         total_experience += experience_months
#         extracted_data["Experience"].add(
#             f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
#         )
#     except Exception:
#         continue  # Skip parsing errors

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)


# # Convert total experience to years & months
# if total_experience > 0:
#     years, months = divmod(total_experience, 12)
#     extracted_data["Total Experience"] = f"{years} years {months} months"

# # Display the cleaned-up results
# print("\n---- Extracted Information ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {','.join(values)}")


KeyError: 'Designation'

In [ ]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Experience + Skills)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# Load the trained NER model
nlp = spacy.load("output/model-best")  # Ensure this path is correct

# Extract text from the resume PDF
text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
doc = nlp(text)

# Store extracted entities dynamically
extracted_data = {
    "Name": set(),
    "College Name": set(),
    "Location": set(),
    "Skills": set(),
    # "Experience": set(),
}

# Extract NER entities
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# Step 1: Extract raw job experience using regex
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

# Step 2: Extract and merge overlapping experience durations
raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())

        raw_date_ranges.append((start_date, end_date))

        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Step 3: Merge overlapping date ranges
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Step 4: Calculate total merged experience
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# Step 5: Rule-based skill matching
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science",
    "mern stack", "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# Display extracted results
print("\n---- Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(values))



---- Extracted Resume Information ----
Name:
  - Muhammad Fareed
  - MUHAMMAD FAREED
College Name:
  - Hamdard University
Skills:
  - MERN Stack
  - PostgreSQL
  - MERN stack
  - Python
  - FAST API
  - React
  - Solidity
  - JavaScript
  - Firebase
Degree:
  - Web and App Development
  - Bachelor in Computer Science
Total Experience:
  - 3 years 6 months


In [ ]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Experience + Skills)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# Load the trained NER model
nlp = spacy.load("output/model-best")  # Ensure this path is correct

# Extract text from the resume PDF
text = extract_text("/content/Abdullah_CV (2).pdf")
doc = nlp(text)

# Store extracted entities dynamically
extracted_data = {
    "Name": set(),
    "College Name": set(),
    "Location": set(),
    "Skills": set(),
    # "Experience": set(),
}

# Extract NER entities
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# Step 1: Extract raw job experience using regex
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

# Step 2: Extract and merge overlapping experience durations
raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())

        raw_date_ranges.append((start_date, end_date))

        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Step 3: Merge overlapping date ranges
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Step 4: Calculate total merged experience
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# Step 5: Rule-based skill matching
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science",
    "mern stack", "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# Display extracted results
print("\n---- Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(values))



---- Extracted Resume Information ----
Name:
  - MUHAMMAD ABDULLAH
College Name:
  - at Hamdard University
  - Hamdard University
Skills:
  - Python
  - Data Science
  - Machine Learning
  - data science
Email Address:
  - abdullah- 13859617a/
  - github.com/M-Abdullah786
  - muhammad-
  - syed-
  - /in/
Degree:
  - Undergraduate student
  - Bachelor of Science - Computer Science
-
  - Bachelor’s student majoring in Computer Science
Total Experience:
  - 0 years 3 months


In [ ]:
# import spacy
# import re
# from pdfminer.high_level import extract_text
# from dateutil import parser
# from datetime import datetime

# # Load the trained NER model
# nlp = spacy.load("output/model-last")

# # Extract text from the resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary (Use Sets to Avoid Duplicates)
# extracted_data = {
#     "Name": set(),
#     "Degree": set(),
#     "College Name": set(),
#     "Location": set(),
#     "Skills": set(),
#     "Experience": set(),
# }

# # Extract Named Entities (NER)
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text.strip())

# # Remove unwanted terms from Skills & Degree
# invalid_terms = {"SKILLS", "SUMMARY", "WORK EXPERIENCE", "EDUCATION"}
# extracted_data["Degree"] -= invalid_terms
# extracted_data["Skills"] -= invalid_terms

# # Extract skills manually (if missing from NER)
# skills_section = re.findall(
#     r"(?i)(?:Skills|Technical Skills|Expertise|SKILLS SUMMARY)[\s\S]*?(?=\n\n|\Z)", text
# )
# if skills_section:
#     skills = re.findall(r"\b[A-Za-z#.\-+]+\b", skills_section[0])
#     extracted_data["Skills"].update(skill for skill in skills if skill.upper() not in invalid_terms)

# # Extract job experience details (using regex)
# experience_pattern = re.findall(
#     r"([\w\s\-/]+)\s*[\n]*\s*(\w+\s*\d{4})\s*–\s*(\w+\s*\d{4}|Present)", text
# )

# total_experience_months = 0
# for job_title, start_date, end_date in experience_pattern:
#     try:
#         start_date = parser.parse(start_date)
#         end_date = (
#             parser.parse(end_date) if "Present" not in end_date else datetime.today()
#         )
#         experience_months = (end_date.year - start_date.year) * 12 + (
#             end_date.month - start_date.month
#         )
#         total_experience_months += experience_months

#         extracted_data["Experience"].add(
#             f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
#         )
#     except Exception:
#         continue  # Skip parsing errors

# # Convert total experience to years & months
# if total_experience_months > 0:
#     years, months = divmod(total_experience_months, 12)
#     extracted_data["Total Experience"] = f"{years} years {months} months"

# # Remove duplicates & format results properly
# for key in extracted_data:
#     extracted_data[key] = list(set(extracted_data[key]))  # Convert sets to lists

# # Display the cleaned-up results
# print("\n---- Extracted Information ----")
# for key, values in extracted_data.items():
#     if values:
#          print(f"{key}: {' '.join(values)}")



---- Extracted Information ----
Name: MUHAMMAD FAREED Muhammad Fareed
Degree: Bachelor in Computer Science
College Name: Hamdard University Saylani Mass & IT Training Program (SMIT)
Web and App Development
Location: Karachi S T M E
Experience: WORK EXPERIENCE

Frontend Developer / Elearning Avenue

Oc (Apr 2022 - May 2023)
Total Experience: h   t y 1 o m n r s e a


In [ ]:
# print("---- Full Resume Text ----")
# print(text)


---- Full Resume Text ----
MUHAMMAD FAREED
M E R N   S T A C K   D E V E L O P E R

Muhammad Fareed / LinkedIn
mfareed1947 / github

Hamdard University
Bachelor in Computer Science (7 semester)
Saylani Mass & IT Training Program (SMIT)
Web and App Development

EDUCATION

Email: mfareed1504@gmail.com
Mobile: +92 3043686779

Karachi, Pakistan
Nov 2021 – Present
Karachi, Pakistan
Jan 2021 – 2022

• Languages:

JavaScript, Python, Typescript , Solidity , Node Js , PostgreSQL

• Frameworks:

React Js, Next Js, Nuxt Js, Express Js, FAST API, Gatsby Js

• Tools:

Tailwind CSS, MUI, Shadcn UI, Style component

SKILLS SUMMARY

• Platforms:

MongoDB, Weaviate, Pinecone, Sanity, Supabase, Firebase, Strapi, Stripe, Redux, Drizzle, Neon Js, Shopify

• Soft Skills:

Rapport Building, Strong Stakeholder Management, People Management, Excellent Communication

MERN Stack Developer / Esspfa IT Solution

July 2023 –Present

◦ Collaborated with cross-functional teams to create intuitive and visually appea

In [ ]:
# import sys , fitz

In [ ]:
# fname = '/content/CV-Parsing-using-Spacy-3/data/test/Muhammad Raza Resume (1).pdf'
# doc = fitz.open(fname)
# text = ""
# for page in doc:
  # text = text + str(page.get_text())
# doc.close()

In [ ]:
# text = text.strip()

In [ ]:
# txt = " ".join(text.split('\n'))

In [ ]:
# print(txt)

In [ ]:
# doc = nlp(text)
# for ent in doc.ents:
  # print(ent.text, '---->' , ent.label_)

In [ ]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-last/ (stored 0%)
  adding: content/output/model-last/transformer/ (stored 0%)
  adding: content/output/model-last/transformer/model (deflated 13%)
  adding: content/output/model-last/transformer/cfg (stored 0%)
  adding: content/output/model-last/vocab/ (stored 0%)
  adding: content/output/model-last/vocab/strings.json (deflated 76%)
  adding: content/output/model-last/vocab/lookups.bin (stored 0%)
  adding: content/output/model-last/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-last/vocab/vectors (deflated 45%)
  adding: content/output/model-last/vocab/key2row (stored 0%)
  adding: content/output/model-last/ner/ (stored 0%)
  adding: content/output/model-last/ner/model (deflated 8%)
  adding: content/output/model-last/ner/moves (deflated 74%)
  adding: content/output/model-last/ner/cfg (deflated 33%)
  adding: content/output/model-last/tokenizer (deflated 81%)
  adding: content/output/model-last/confi

In [ ]:
# !python -m spacy debug data config.cfg


✘ Config validation error
disabled	field required
tokenizer	field required
before_creation	field required
after_creation	field required
after_pipeline_creation	field required
batch_size	field required
{'lang': 'en', 'pipeline': ['transformer', 'ner'], 'vectors': {'@vectors': 'spacy.Vectors.v1'}}

If your config contains missing values, you can run the 'init fill-config'
command to fill in all the defaults, if possible:

python -m spacy init fill-config config.cfg config.cfg 



---
<h4>For Reference:</h4>
<a href='https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git'>CV-Parsing-using-Spacy-3
</a><br>
<a href='https://github.com/yashlikescode/spacyResumeParcer.git'>spacyResumeParcer</a>
<hr>